In [2]:
from config import *
from api_handler import *
from db_handler import  *
import pandas as pd

### Setting up endpoint

In [4]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    'Access': 'application/json',
    'Authorization': f'Bearer {YELP_API_KEY}',
}
params = {
    'latitude': LATITUDE_YELP,
    'longitude': LONGITUDE_YELP,
    'limit': LIMIT_YELP,
    'radius': RADIUS_YELP,
    'categories': CATEGORIES_YELP,
    'sort_by': SORT_BY_YELP,
    
    }

In [ ]:
response = get_req(url, params=params, headers=headers)

### Preparing data to insert into DB

In [53]:
data_to_insert = [(result.get('id', None), 
                      result.get('name', None),
                       result.get('rating', None),
                        result.get('review_count', None),
                         result.get('url', None),
                          ' '.join(result.get('location', {}).get('display_address', '')))
                               for result in response['businesses']]

In [55]:
rows_to_insert = ''
for new_query in data_to_insert:
    if new_query == data_to_insert[-1]:
        rows_to_insert += str(new_query)
    else:
        rows_to_insert += str(new_query) + ','

### Creating Tables & Inserting Data

In [3]:
connection = create_connection("./data/yelp_coctail_spots.sqlite")

In [4]:
create__table = """
CREATE TABLE IF NOT EXISTS yelp_coctail_spots (
  PK_ID INTEGER PRIMARY KEY AUTOINCREMENT,
  id TEXT NOT NULL,
  name TEXT NOT NULL,
  rating REAL,
  url TEXT,
  location TEXT,
  review_count INTEGER
);
"""

In [5]:
execute_query(connection, create__table)

In [67]:
create_spots = f"""
INSERT INTO
  yelp_coctail_spots (id, name, rating, url, location, review_count)
VALUES
    {rows_to_insert};
"""

In [70]:
execute_query(connection, create_spots)

### Requesting DB

In [7]:
select_coctail_spots = "SELECT * from yelp_coctail_spots"
coctail_spots = execute_read_query(connection, select_coctail_spots)

In [16]:
df = pd.DataFrame(coctail_spots, 
                  columns=['PKID','id', 'name', 'rating', 'review_count', 'url', 'location'])

In [17]:
df = df[['name', 'rating', 'location']]

In [18]:
df.sort_values('rating', ascending=False).head(10).to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name</th>\n      <th>rating</th>\n      <th>location</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>28</th>\n      <td>Le Speakeasy</td>\n      <td>5.0</td>\n      <td>120 Rue McGill Montreal, QC H2Y 2E5 Canada</td>\n    </tr>\n    <tr>\n      <th>27</th>\n      <td>Terrasse W</td>\n      <td>5.0</td>\n      <td>901 Rue du Square-Victoria Montreal, QC H2Z 1R1 Canada</td>\n    </tr>\n    <tr>\n      <th>26</th>\n      <td>Terrasse des Sucres</td>\n      <td>5.0</td>\n      <td>106 Rue Saint-Paul O 5th Floor Montreal, QC H2Y 1Z3 Canada</td>\n    </tr>\n    <tr>\n      <th>25</th>\n      <td>Notkins</td>\n      <td>5.0</td>\n      <td>1101 Rue de Bleury Montreal, QC H2Z 1N1 Canada</td>\n    </tr>\n    <tr>\n      <th>24</th>\n      <td>La Croisette</td>\n      <td>5.0</td>\n      <td>2 Rue de la Commune O Montreal, QC H2Y 4B2 Canada</td>\n    </tr>\n    <tr>\n    